In [1]:
# Dependencies and Setup
import pandas as pd
from sqlalchemy import create_engine

#from config import username, password, host, port

In [2]:
#Read Streaming Platform CSV into dataframe
streaming_csv="DataSets/MoviesOnStreamingPlatforms_updated.csv"
streaming_data_df=pd.read_csv(streaming_csv, index_col=[0])
streaming_data_df.head()

,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [3]:
#Read IMDB CSV into dataframe
imdb_csv="DataSets/movie_metadata.csv"
imdb_data_df=pd.read_csv(imdb_csv, encoding="utf-8")
imdb_data_df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
#Clean IMDB_DF 
new_imdb_df=imdb_data_df[["Title","gross", "budget"]].copy()
new_imdb_df.head()

,Title,gross,budget
0,Avatar,760505847.0,237000000.0
1,Pirates of the Caribbean: At World's End,309404152.0,300000000.0
2,Spectre,200074175.0,245000000.0
3,The Dark Knight Rises,448130642.0,250000000.0
4,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN


In [5]:
#Clean Streaming_DF 
new_streaming_df=streaming_data_df.drop(columns=["ID", "Age", "Rotten Tomatoes", "Type", "Runtime"]).copy()
new_streaming_df.head()

,Title,Year,IMDb,Netflix,Hulu,Prime Video,Disney+,Directors,Genres,Country,Language
0,Inception,2010,8.8,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French"
1,The Matrix,1999,8.7,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English
2,Avengers: Infinity War,2018,8.5,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English
3,Back to the Future,1985,8.5,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English
4,"The Good, the Bad and the Ugly",1966,8.8,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian


In [6]:
#Set up engine and create connection
sql_lite = "sqlite:///my_sqlite.db"
engine = create_engine(sql_lite)

In [7]:
#Create IMDB table in SQLITE DB
new_imdb_df.to_sql(name="new_imdb_df", con=engine, if_exists="append", index=False)

In [8]:
#Read Streaming table in SQLITE DB
pd.read_sql_query('select * from new_imdb_df', con=engine).head()

,Title,gross,budget
0,Avatar,760505847.0,237000000.0
1,Pirates of the Caribbean: At World's End,309404152.0,300000000.0
2,Spectre,200074175.0,245000000.0
3,The Dark Knight Rises,448130642.0,250000000.0
4,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN


In [9]:
#Create Streaming table in SQLITE DB
new_streaming_df.to_sql(name="new_streaming_df", con=engine, if_exists="append", index=False)

In [10]:
#Read Streaming table in SQLITE DB
pd.read_sql_query('select * from new_streaming_df', con=engine)

,Title,Year,IMDb,Netflix,Hulu,Prime Video,Disney+,Directors,Genres,Country,Language
0,Inception,2010,8.8,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French"
1,The Matrix,1999,8.7,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English
2,Avengers: Infinity War,2018,8.5,1,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English
3,Back to the Future,1985,8.5,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English
4,"The Good, the Bad and the Ugly",1966,8.8,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian
...,...,...,...,...,...,...,...,...,...,...,...
16739,The Ghosts of Buxley Hall,1980,6.2,0,0,0,1,Bruce Bilson,"Comedy,Family,Fantasy,Horror",United States,English
16740,The Poof Point,2001,4.7,0,0,0,1,Neal Israel,"Comedy,Family,Sci-Fi",United States,English
16741,Sharks of Lost Island,2013,5.7,0,0,0,1,Neil Gelinas,Documentary,United States,English
16742,Man Among Cheetahs,2017,6.6,0,0,0,1,Richard Slater-Jones,Documentary,United States,English


In [11]:
#save sql join as combined table
combined_table = pd.read_sql("SELECT * FROM new_streaming_df INNER JOIN new_imdb_df ON new_streaming_df.Title = new_imdb_df.Title ", con=engine)

#Create combined table in SQLITE DB
combined_table.to_sql(name="combined_table", con=engine, if_exists="append", index=False)

In [12]:
#Check SQLDatabase to ensure creation of tables
engine.table_names()

['combined_table', 'new_imdb_df', 'new_streaming_df']

In [13]:
#ensure combined table reads correctly
pd.read_sql('select * from combined_table', con=engine)

,Title,Year,IMDb,Netflix,Hulu,Prime Video,Disney+,Directors,Genres,Country,Language,gross,budget
0,Inception,2010,8.8,1,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",292568851.0,160000000.0
1,The Matrix,1999,8.7,1,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,171383253.0,63000000.0
2,Back to the Future,1985,8.5,1,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,210609762.0,19000000.0
3,"The Good, the Bad and the Ugly",1966,8.8,1,0,1,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,6100000.0,1200000.0
4,The Pianist,2002,8.5,1,0,1,0,Roman Polanski,"Biography,Drama,Music,War","United Kingdom,France,Poland,Germany","English,German,Russian",32519322.0,35000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,Shipwrecked,1990,6.7,0,0,0,1,Nils Gaup,"Adventure,Family","Norway,Sweden,United States",English,NaN,60000000.0
1045,The Country Bears,2002,4.1,0,0,0,1,Peter Hastings,"Comedy,Family,Music,Musical",United States,English,16988996.0,35000000.0
1046,Doug's 1st Movie,1999,5.0,0,0,0,1,Maurice Joyce,"Animation,Adventure,Comedy,Drama,Family,Fantas...",United States,English,19421271.0,5000000.0
1047,Meet the Deedles,1998,4.1,0,0,0,1,Steve Boyum,"Comedy,Family",United States,English,4308981.0,24000000.0


In [14]:
#save as new variable
new_combine = pd.read_sql('select * from combined_table', con=engine)

In [31]:
#save as html file
combined_html = combined.to_html

In [32]:
#rename html variable
data = combined_html()

In [34]:
#export as an html file for website
export_file = open("data.html","w")
export_file.write(data)
export_file.close()